In [1]:
from pathlib import Path
import pandas as pd

path_input_csv = Path("../../input/santa-2024/sample_submission.csv")
path_save = Path("./save")
path_save.mkdir(parents=True, exist_ok=True)
path_model = Path("../../input/gemma-2/")

df = pd.read_csv(path_input_csv)

In [2]:
from evaluation import PerplexityCalculator

calculator = PerplexityCalculator(model_path=str(path_model))

/home/khorie/anaconda3/envs/kaggle3.11-3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]


In [3]:
from util import get_path_words_best

import copy

import numpy as np
import random

n_idx_total = len(df)
list_words_best = []
list_perplexity_best = []

flag_shuffle = False

for idx in range(n_idx_total):
    if flag_shuffle:
        text = df.iloc[idx, 1]
        list_words = text.split()
        random.shuffle(list_words)
    else:
        score, list_words = get_path_words_best(idx)
    text = " ".join(list_words)
    list_words_best.append(copy.deepcopy(list_words))
    list_perplexity_best.append(calculator.get_perplexity(text))


In [4]:
# dicts_perplexity_memo = [{} for _ in range(n_idx_total)]

In [5]:
import copy

import numpy as np
import random

# n_idx = 2
# text = df.loc[n_idx, "text"]


# select random word and insert random place
def make_neighbor_1(words):
    # ランダムな単語を選択し、ランダムな場所に挿入
    idx = random.randint(0, len(words) - 1)
    word = words[idx]
    # remove
    words.pop(idx)
    # insert
    idx_insert = random.randint(0, len(words))
    words.insert(idx_insert, word)
    return words


# select random word sequence and insert random place
def make_neighbor_2(words):
    idx1 = random.randint(0, len(words))
    idx2 = random.randint(0, len(words))
    if idx1 == idx2:
        return None
    if idx1 > idx2:
        idx1, idx2 = idx2, idx1
    words_mid = words[idx1:idx2]
    words = words[:idx1] + words[idx2:]

    idx_insert = random.randint(0, len(words))
    words = words[:idx_insert] + words_mid + words[idx_insert:]
    return words


# select random word sequence and insert at the beginning or the end
def make_neighbor_3(words):
    idx1 = random.randint(1, len(words))
    idx2 = random.randint(1, len(words))
    if idx1 == idx2:
        return None
    if idx1 > idx2:
        idx1, idx2 = idx2, idx1
    words1 = words[:idx1]
    words2 = words[idx1:idx2]
    words3 = words[idx2:]
    coin = random.randint(1, 2)
    if coin == 1:
        words = words1 + words3 + words2
    else:
        words = words2 + words1 + words3
    return words


# rotate entire
def make_neighbor_4(words):
    # rotate entire
    idx = random.randint(1, len(words) - 1)
    words = words[idx:] + words[:idx]
    return words


# swap adjacent
def make_neighbor_5(words):
    idx = random.randint(0, len(words) - 2)
    words[idx], words[idx + 1] = words[idx + 1], words[idx]
    return words


list_prob = {
    1: 10,
    2: 50,
    3: 10,
    4: 0,
    5: 1,
}

prob_total = sum(list_prob.values())
for key in list_prob.keys():
    list_prob[key] = list_prob[key] / prob_total


def make_neighbor(words_input):
    words_return = None
    while words_return is None:
        words = copy.deepcopy(words_input)
        # list_coin = [1, 2, 3, 4, 5]
        # coin = random.randint(1, 5)
        coin = np.random.choice(list(list_prob.keys()), p=list(list_prob.values()))

        if coin == 1:
            func_neighbor = make_neighbor_1
        elif coin == 2:
            func_neighbor = make_neighbor_2
        elif coin == 3:
            func_neighbor = make_neighbor_3
        elif coin == 4:
            func_neighbor = make_neighbor_4
        elif coin == 5:
            func_neighbor = make_neighbor_5
        else:
            raise ValueError

        words_return = func_neighbor(words)

        if words_return == words_input:
            words_return = None

    assert sorted(words_input) == sorted(words_return)

    return words_return


iter_total = 100
iter_now = 0

n_sample = 32

# words_best = text.split(" ")
# perplexity_best = calculator.get_perplexity(" ".join(words_best))

# print(f"original:{perplexity_best:.2f}")
# print(f" words_best:{words_best}")


def hillclimbing(
    n_idx, words_best, perplexity_best, iter_total=100, n_sample=n_sample, verbose=0
):
    iter_now = 0

    while iter_now < iter_total:
        list_words_nxt = []
        list_texts_nxt = []
        for _ in range(n_sample):
            words_nxt = make_neighbor(words_best)
            list_words_nxt.append(words_nxt)
            list_texts_nxt.append(" ".join(words_nxt))

        list_perplexity_nxt = calculator.get_perplexity(list_texts_nxt)
        # take min
        idx_min = np.argmin(list_perplexity_nxt)
        words_nxt = list_words_nxt[idx_min]
        # perplexity_nxt = list_perplexity_nxt[idx_min]
        perplexity_nxt = calculator.get_perplexity(" ".join(words_nxt))

        if iter_now % 50 == 0:
            if verbose:
                print(
                    f"iter:{iter_now} best:{perplexity_best:.2f} nxt:{perplexity_nxt:.2f}"
                )
                # print(f" words_best:{words_best}")

        if perplexity_nxt < perplexity_best:
            perplexity_best = perplexity_nxt
            # perplexity_best_recalculated = calculator.get_perplexity(" ".join(words_nxt))
            # print(list_perplexity_nxt)
            # print(
            #     f"iter:{iter_now} best:{perplexity_best:.2f} recalculated:{perplexity_best_recalculated:.2f}"
            # )
            words_best = copy.deepcopy(words_nxt)
            # print("accepted")

        iter_now += 1

    return words_best, perplexity_best


def save_text(n_idx, text, verbose=0):
    path_save_idx = path_save / f"{n_idx:04d}"
    if not path_save_idx.exists():
        path_save_idx.mkdir()
    score = calculator.get_perplexity(text)
    if verbose:
        print(f"score:{score:.4f}")
    path_save_text = path_save_idx / f"{score:.4f}.txt"

    with path_save_text.open("w") as f:
        f.write(text)


In [6]:
list_idx_randomize = []
for idx in list_idx_randomize:
    text = df.iloc[idx, 1]
    list_words = text.split()
    random.shuffle(list_words)
    text = " ".join(list_words)
    list_words_best[idx] = copy.deepcopy(list_words)
    list_perplexity_best[idx] = calculator.get_perplexity(text)

In [7]:
list_batch_size = [64, 100, 80, 40, 30, 16]

list_no_update_cnt = [0] * n_idx_total
list_num_kick = [0] * n_idx_total

max_no_update_cnt = 1


def calc_n_kick(n_idx):
    n_kick = list_num_kick[n_idx]
    i = 1
    while True:
        if n_kick >= i:
            n_kick = n_kick - i
        else:
            break
        i += 1
    n_kick = i - n_kick
    # n_kick -= 1  # to make the final hill climbing longer
    n_kick = n_kick * 2 - 2
    return n_kick


def ILS_kick(words, n_kick=2):
    for _ in range(n_kick):
        words = copy.deepcopy(words)
        words = make_neighbor(words)
    return words

In [8]:
import gc
import torch


def free_memory():
    gc.collect()
    torch.cuda.empty_cache()

In [9]:
n_idx = 0
list_idx_skip = [0]
while True:
    if n_idx == n_idx_total:
        n_idx = 0

    if n_idx in list_idx_skip:
        n_idx += 1
        continue

    free_memory()

    words_best = list_words_best[n_idx]
    perplexity_best_bef = list_perplexity_best[n_idx]

    print(f"#" * 80)
    print(f"n_idx:{n_idx} perplexity_best:{perplexity_best_bef:.2f}")
    words_best, perplexity_best = hillclimbing(
        n_idx,
        words_best,
        perplexity_best_bef,
        iter_total=100,
        n_sample=list_batch_size[n_idx],
        verbose=1,
    )
    print(f"n_idx:{n_idx} perplexity_best:{perplexity_best:.2f}")

    did_kick = False
    if perplexity_best_bef == perplexity_best:
        list_no_update_cnt[n_idx] += 1
        if list_no_update_cnt[n_idx] >= max_no_update_cnt:
            n_kick = calc_n_kick(n_idx)
            print(f"apply {n_kick} kick")
            list_num_kick[n_idx] += 1
            did_kick = True
            list_no_update_cnt[n_idx] = 0
            words_best = ILS_kick(words_best, n_kick=n_kick)
            perplexity_best = calculator.get_perplexity(" ".join(words_best))
    else:
        list_no_update_cnt[n_idx] = 0

    list_words_best[n_idx] = words_best
    list_perplexity_best[n_idx] = perplexity_best

    if not did_kick:
        save_text(n_idx, " ".join(words_best), verbose=1)

    n_idx += 1


################################################################################
n_idx:1 perplexity_best:475.40
iter:0 best:475.40 nxt:550.83
iter:50 best:475.40 nxt:564.77
n_idx:1 perplexity_best:475.40
apply 0 kick
################################################################################
n_idx:2 perplexity_best:299.23
iter:0 best:299.23 nxt:355.71
iter:50 best:299.23 nxt:364.43
n_idx:2 perplexity_best:299.23
apply 0 kick
################################################################################
n_idx:3 perplexity_best:251.46
iter:0 best:251.46 nxt:306.36
iter:50 best:251.46 nxt:328.71
n_idx:3 perplexity_best:251.46
apply 0 kick
################################################################################
n_idx:4 perplexity_best:98.49
iter:0 best:98.49 nxt:115.78
iter:50 best:98.49 nxt:100.68
n_idx:4 perplexity_best:98.49
apply 0 kick
################################################################################
n_idx:5 perplexity_best:98.21
iter:0 best:98.21 nxt:111

In [ ]:
assert False

In [ ]:
def save_text(n_idx, text):
    path_save_idx = path_save / f"{n_idx:04d}"
    if not path_save_idx.exists():
        path_save_idx.mkdir()
    score = calculator.get_perplexity(text)
    print(f"score:{score:.4f}")
    path_save_text = path_save_idx / f"{score:.4f}.txt"

    with path_save_text.open("w") as f:
        f.write(text)


save_text(n_idx, " ".join(words_best))

In [ ]:
perplexity_best, perplexity_nxt

In [ ]:
words_best

In [ ]:
import random

random.randint(0, 1)

In [ ]:
# print number of words in each row
for n_idx, row in df.iterrows():
    print(f"row {n_idx}: {len(row['text'].split())} words")


In [ ]:
n_idx = 0
text = df.loc[n_idx, "text"]
list_n_space = []
# list_score = []

list_texts = []

for n_space in range(3, 303, 1):
    text_new = (" " * n_space).join(text.split())
    list_texts.append(text_new)
    # score = calculator.get_perplexity(text_new)
    # # print(f"n_space: {n_space} score: {score}")
    list_n_space.append(n_space)
    # list_score.append(score)

list_score = calculator.get_perplexity(list_texts, batch_size=10)

# df_score = pd.DataFrame({"n_space": list_n_space, "score": list_score})

In [ ]:
import matplotlib.pyplot as plt

plt.plot(list_n_space, list_score)
plt.xlabel("n_space")
plt.ylabel("score")
# log
plt.yscale("log")

In [ ]:
for n_space in range(3, 100, 10):
    df_submission = df.copy()
    for n_idx, row in df_submission.iterrows():
        text = row["text"]
        words = text.split()
        text = (" " * n_space).join(words)
        df_submission.iloc[n_idx, 1] = text
    df_submission.to_csv(path_save / f"submission_{n_space}.csv", index=False)
